In [1]:
%pip install geopandas

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import fiona

# Set the SHAPE_RESTORE_SHX option
fiona.drvsupport.supported_drivers['ESRI Shapefile'] = 'rw'
fiona.drvsupport.supported_drivers['SHAPE_RESTORE_SHX'] = 'YES'

# Read the shapefile
shapefile_path = "/content/SA1_2021_AUST_GDA2020.shp"
total_rows = len(gpd.read_file(shapefile_path))
print(f"Total number of rows in the shapefile: {total_rows}")

# Read the CSV file
csv_path = "/content/houses-sample.csv"
houses_df = pd.read_csv(csv_path)
print(houses_df.head())  # Print the first few rows of the CSV file


Total number of rows in the shapefile: 61845
   ADDRESS_DEFAULT_GEOCODE_PID DATE_CREATED  DATE_RETIRED ADDRESS_DETAIL_PID  \
0                   3005984401    1/11/2012           NaN     GANSW705069147   
1                   3005829639    1/11/2012           NaN     GANSW704998680   
2                      2397791    1/11/2012           NaN     GAVIC424489441   
3                      3195931    1/11/2012           NaN     GAVIC420680970   
4                   3001407829    5/02/2015           NaN     GAWA_147486543   

  GEOCODE_TYPE_CODE   LONGITUDE   LATITUDE ADDED-COLUMN-STATE  
0                PC  150.929034 -33.703897                NSW  
1                PC  151.030107 -34.003832                NSW  
2               FCS  141.337645 -34.388504                VIC  
3               FCS  145.125946 -37.843119                VIC  
4                PC  115.906418 -31.915203                 WA  


In [7]:
import geopandas as gpd
import pandas as pd
from pyproj import CRS
import multiprocessing as mp
import numpy as np

# Load the points from the CSV file
csv_path = "/content/houses-sample.csv"
pnts = pd.read_csv(csv_path)

if 'LONGITUDE' in pnts.columns and 'LATITUDE' in pnts.columns:
    pnts = pnts.rename(columns={'LONGITUDE': 'x', 'LATITUDE': 'y'})

# Convert points to GeoDataFrame
pnts_gdf = gpd.GeoDataFrame(
    pnts,
    geometry=gpd.points_from_xy(pnts.x, pnts.y),
    crs="EPSG:7844"
)

# Project to a suitable projected CRS (GDA2020 / MGA zone 55)
projected_crs = CRS.from_epsg(7844)
pnts_gdf = pnts_gdf.to_crs(projected_crs)

# Read the entire shapefile
map_data = gpd.read_file("/content/SA1_2021_AUST_GDA2020.shp")
map_data = map_data.to_crs(projected_crs)

# Create spatial index for the polygons
spatial_index = map_data.sindex

def process_chunk(chunk):
    # Perform spatial join for this chunk
    pnts_with_area = gpd.sjoin(chunk, map_data, how="left", predicate='within')

    # Identify points without an assigned area
    missing_points = pnts_with_area[pnts_with_area['SA2_NAME21'].isnull()]

    if not missing_points.empty:
        # Perform nearest join for missing points
        nearest_join = gpd.sjoin_nearest(missing_points[['geometry']], map_data, how="left")

        # Update the main result with nearest join results
        pnts_with_area.loc[missing_points.index, 'SA2_NAME21'] = nearest_join['SA2_NAME21']
        pnts_with_area.loc[missing_points.index, 'SA2_CODE21'] = nearest_join['SA2_CODE21']

    # Select and rename relevant columns
    result = pnts_with_area[['x', 'y', 'SA2_NAME21', 'SA2_CODE21']]
    result = result.rename(columns={'SA2_NAME21': 'area', 'SA2_CODE21': 'area_code'})
    return result

# Split points into chunks for parallel processing
num_cores = mp.cpu_count()
chunks = np.array_split(pnts_gdf, num_cores)

# Parallel processing
with mp.Pool(num_cores) as pool:
    all_results = pool.map(process_chunk, chunks)

# Combine all results
final_result = pd.concat(all_results, ignore_index=True)

# Check if there are still any missing areas
if final_result['area'].isnull().any() or final_result['area_code'].isnull().any():
    print("Warning: Some points were not assigned to an SA2 area even after nearest join. Please check your data.")

print(final_result)

# Merge the final_result with the original points DataFrame to add area and area_code
houses_with_areas = pd.merge(pnts, final_result, on=['x', 'y'], how='left')

# Save the result to a CSV file
output_csv_path = "/content/houses_with_areas.csv"
houses_with_areas.to_csv(output_csv_path, index=False)

print(f"CSV file saved to: {output_csv_path}")

# Display the first few rows of the resulting DataFrame
print(houses_with_areas.head())

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/usr/local/l

                 x          y                            area  area_code
0       150.929034 -33.703897    Kellyville Ridge - The Ponds  116021628
1       151.030107 -34.003832        Illawong - Alfords Point  128021534
2       141.337645 -34.388504               Mildura Surrounds  215021398
3       145.125946 -37.843119                        Box Hill  207031163
4       115.906418 -31.915203  Bayswater - Embleton - Bedford  504011045
...            ...        ...                             ...        ...
749995  151.298392 -33.736766     Collaroy - Collaroy Plateau  122031693
749996  153.430861 -28.017497        Surfers Paradise - South  309101562
749997  115.602548 -31.509019                       Two Rocks  505031257
749998  144.942850 -37.821223                       Docklands  206041118
749999  145.684255 -16.862956                        Redlynch  306011141

[750000 rows x 4 columns]
CSV file saved to: /content/houses_with_areas.csv
   ADDRESS_DEFAULT_GEOCODE_PID DATE_CREATED  DA

In [9]:
# Count the number of missing values in the 'area' column
missing_area_count = final_result['area'].isnull().sum()

print(f"Number of missing values in the 'area' column: {missing_area_count}")


Number of missing values in the 'area' column: 0


In [10]:
# Merge the final_result with the original points DataFrame to add area and area_code
houses_with_areas = pd.merge(pnts, final_result, on=['x', 'y'], how='left')

# Display the resulting DataFrame with the new columns
houses_with_areas

output_csv_path = "/content/houses_with_areas2.csv"
houses_with_areas.to_csv(output_csv_path, index=False)

print(f"CSV file saved to: {output_csv_path}")

# Display the first few rows of the resulting DataFrame
houses_with_areas.head()

KeyboardInterrupt: 

In [11]:
houses_with_areas.head()

,ADDRESS_DEFAULT_GEOCODE_PID,DATE_CREATED,DATE_RETIRED,ADDRESS_DETAIL_PID,GEOCODE_TYPE_CODE,x,y,ADDED-COLUMN-STATE,area,area_code
0,3005984401,1/11/2012,NaN,GANSW705069147,PC,150.929034,-33.703897,NSW,Kellyville Ridge - The Ponds,116021628
1,3005829639,1/11/2012,NaN,GANSW704998680,PC,151.030107,-34.003832,NSW,Illawong - Alfords Point,128021534
2,2397791,1/11/2012,NaN,GAVIC424489441,FCS,141.337645,-34.388504,VIC,Mildura Surrounds,215021398
3,3195931,1/11/2012,NaN,GAVIC420680970,FCS,145.125946,-37.843119,VIC,Box Hill,207031163
4,3001407829,5/02/2015,NaN,GAWA_147486543,PC,115.906418,-31.915203,WA,Bayswater - Embleton - Bedford,504011045


In [ ]:
print(pnts_gdf.crs)
print(map_chunk.crs)

EPSG:7844
EPSG:7844
